In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
df = pd.DataFrame()
for year in range(2018,2023):
    yr_df = pd.read_pickle(f'./data/{year}_matches.pkl') # load df for year

    # Add moving averages
    teams = np.unique(np.append(yr_df['Home Team'], yr_df['Away Team']))
    for team in teams:
        team_df = yr_df[(yr_df['Home Team']==team) | (yr_df['Away Team']==team)]
        assert team_df.shape[0] == 38
        

    # df = pd.concat([df, ], ignore_index=True, axis=0)

    break

In [ ]:
for i in yr_df.index:
    hteam = yr_df.loc[i,'Home Team']
    ateam = yr_df.loc[i,'Away Team']

    for key, team in [('Home',hteam), ('Away',ateam)]:
        n = 5
        team_recent_df = yr_df[(yr_df['Home Team']==team) | (yr_df['Away Team']==team)].iloc[-n-1:-1,:]
        team_recent_goals = (
            ((team_recent_df['Home Team']==team) * team_recent_df['Home Goals']) 
            + ((team_recent_df['Away Team']==team) * team_recent_df['Away Goals'])
            )
        team_recent_xg = (
            ((team_recent_df['Home Team']==team) * team_recent_df['Home xG']) 
            + ((team_recent_df['Away Team']==team) * team_recent_df['Away xG'])
            )

In [ ]:
i, team

In [ ]:
team_recent_df[['Date', 'Matchweek', 'Home Team', 'Away Team', 'Home Goals', 'Away Goals', 'Home xG', 'Away xG']]

In [ ]:
team_goal_trend

In [ ]:
n = 5
df.groupby('Home Team').rolling(n)['Home Goals'].mean()

In [ ]:
train, test = train_test_split(df, train_size=0.7)
train, validation = train_test_split(train, train_size=0.7)